In [1]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

### IMPORT CSV FILE (REQUESTS)

In [2]:
import requests
r = requests.get('https://docs.google.com/spreadsheets/d/1DSHrCzgdbTgasnsxzJEloAT5wnH3u0EH4ZKwnhLnkVU/pub?output=csv')
data = r.content

### READ CSV FILES (PANDAS) + FORMAT DATE

In [3]:
#r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp'] if columns not specififed
#df = pd.read_csv('Q:\\Sam\\Untitled spreadsheet - Sheet1.csv')
df = pd.read_csv(io.BytesIO(data), infer_datetime_format=True,
                           parse_dates=['Date'],
                           dayfirst=True )


### REVIEW DATA

In [4]:
df.head(2)

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51
0,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,NaN,6,12,...,191.5,1.93,1.81,2.01,1.97,1.93,1.93,1.94,1.88,NaN
1,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,NaN,10,7,...,198.5,1.93,1.81,1.96,1.93,1.93,1.93,2.04,1.92,NaN


### EXAMINE THE DATAFRAME

df.shape 

df.dtypes()  

df.describe() 

### SLICE THE DATAFRAME

[1:] everything fron index 1 onwards 

[:1] everything up to index 1 

[-5:] everything from the last 5 elements on, aka- only the last 5 elements

[:-5] everything up to fifth last element  

[3:6] between index 3 and 6



df[1:] 

df.ix[:5, :10] (rows, columns) 

df.loc[0].Year 

df.loc[517].head()

Manually slice up a date: df["Month"] = df.Date.str[3:6]

df["Year"] = str(20) + df.Date.str[-2:]

### CREATE FUNCTIONS TO PARSE DATES BY ROW

In [5]:
def add_year (df):
    return str(df['Date'].year)

In [6]:
def add_month (df):
    return str(df['Date'].month)

In [7]:
def add_day (df):
    return str(df['Date'].day)

### APPLY FUNCTIONS AND RETURN NEW YEAR/MONTH/DATE COLUMN PER ROW

In [8]:
df["Year"] = df.apply(add_year, axis =1)
df["Month"] = df.apply(add_month, axis =1)
df["Day"] = df.apply(add_day, axis =1)

### TEST DATES

In [9]:
df.head(2)

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,Year,Month,Day
0,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,NaN,6,12,...,2.01,1.97,1.93,1.93,1.94,1.88,NaN,2017,4,16
1,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,NaN,10,7,...,1.96,1.93,1.93,1.93,2.04,1.92,NaN,2017,4,16


### FINDING UNIQUE ELEMENTS

df.Venue.count()

len(df.Venue.unique())

df.Venue.unique()


### DEFINE FUNCTION TO RETURN A DICTIONARY OF CITY NAMES

In [10]:
def get_city(row):
    di_cities= {"MCG": "Melbourne",
     "Gabba": "Brisbane",
     "Etihad Stadium": "Melbourne",
     "Adelaide Oval": "Adelaide",
     "Manuka Oval": "Canberra",
     "Domain Stadium": "Perth",
     "Metricon Stadium": "Gold Coast",
     "Blundstone Arena": "Hobart",
     "SCG": "Sydney",
     "Spotless Stadium": "Homebush",
     "ANZ Stadium": "Homebush",
     "Simonds Stadium": "Geelong",
     "Aurora Stadium": "Launceston",
     "Cazaly’s Stadium": "Cairns",
     "TIO Stadium": "Darwin",
     "Traeger Park": "Alice Springs",
     "Westpac Stadium": "Wellington",
     "AAMI Stadium": "Adelaide",
     "Blacktown Park": "Homebush" }
    return di_cities[row.Venue]

### APPLY FUNCTION AND RETURN IT TO A NEW CITIES COLUMN PER ROW

In [11]:
df["City"] = df.apply(get_city, axis=1)
df.head(2)

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,Year,Month,Day,City
0,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,NaN,6,12,...,1.97,1.93,1.93,1.94,1.88,NaN,2017,4,16,Melbourne
1,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,NaN,10,7,...,1.93,1.93,1.93,2.04,1.92,NaN,2017,4,16,Brisbane


### DEFINE FUNCTION TO RETURN A DICTIONARY OF CITY BASED URLS

In [12]:
#rename station_ids
def get_station_ID(row):
    di_stations= {
         "Brisbane": "334797792&p_stn_num=040913",
         "Melbourne": "1487214643&p_stn_num=086232",
         "Adelaide": "105924303&p_stn_num=023011",
         "Canberra": "986923182&p_stn_num=070246",
         "Perth": "16771239&p_stn_num=009151",
         "Gold Coast": "326713690&p_stn_num=040416",
         "Hobart": "1768351258&p_stn_num=094030",
         "Sydney": "873151344&p_stn_num=066073",
         "Homebush": "871566312&p_stn_num=066013",
         "Geelong": "1519465927&p_stn_num=087162",
         "Launceston": "1658844915&p_stn_num=091072",
         "Cairns": "194935783&p_stn_num=031218",
         "Darwin": "40163856&p_stn_num=014167",
         "Alice Springs": "48632698&p_stn_num=015590",
         "Wellington": "N/A"}
    return di_stations[row.City]

### APPLY FUNCTION AND RETURN IT TO A STATION ID'S COLUMN PER ROW

In [13]:
df["station_ID"] = df.apply(get_station_ID, axis=1)
df.head(2)

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,Year,Month,Day,City,station_ID
0,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,NaN,6,12,...,1.93,1.93,1.94,1.88,NaN,2017,4,16,Melbourne,1487214643&p_stn_num=086232
1,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,NaN,10,7,...,1.93,1.93,2.04,1.92,NaN,2017,4,16,Brisbane,334797792&p_stn_num=040913


### TEST NEW COLUMNS

In [14]:
df.head(2)

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,Year,Month,Day,City,station_ID
0,2017-04-16,15:20,Hawthorn,Geelong,MCG,48,134,NaN,6,12,...,1.93,1.93,1.94,1.88,NaN,2017,4,16,Melbourne,1487214643&p_stn_num=086232
1,2017-04-16,16:40,Brisbane,Richmond,Gabba,67,119,NaN,10,7,...,1.93,1.93,2.04,1.92,NaN,2017,4,16,Brisbane,334797792&p_stn_num=040913


### REORDER DATA FRAME TO SHOW NEW INFORMATION AT THE START

In [15]:
#last 5 columns + all colums less last 5
cols = df.columns.tolist()
new_cols = cols[-5:] + cols[:-5]
df=df[new_cols]

### DEFINE FUNCTION TO RETURN THE URL

In [16]:
def find_url(row):
    Base_Link_01= "http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear="
    Base_Link_02= "&p_c=-"
    link = Base_Link_01 + row.Year + Base_Link_02 + row.station_ID
    return link
    #print (link)

### APPLY FUNCTION AND RETURN IT TO A 'URLS' COLUMN PER ROW

In [17]:
df["URL"] = df[df.City != 'Wellington'].apply(find_url, axis=1)

In [18]:
df.head()

,Year,Month,Day,City,station_ID,Date,Kick Off (local),Home Team,Away Team,Venue,...,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,URL
0,2017,4,16,Melbourne,1487214643&p_stn_num=086232,2017-04-16,15:20,Hawthorn,Geelong,MCG,...,1.93,1.81,2.01,1.97,1.93,1.93,1.94,1.88,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
1,2017,4,16,Brisbane,334797792&p_stn_num=040913,2017-04-16,16:40,Brisbane,Richmond,Gabba,...,1.93,1.81,1.96,1.93,1.93,1.93,2.04,1.92,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
2,2017,4,16,Melbourne,1487214643&p_stn_num=086232,2017-04-16,15:20,Collingwood,St Kilda,Etihad Stadium,...,1.93,1.77,1.91,1.91,1.93,1.93,1.94,1.94,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
3,2017,4,15,Adelaide,105924303&p_stn_num=023011,2017-04-15,19:10,Adelaide,Essendon,Adelaide Oval,...,1.93,1.85,2.17,1.85,1.93,1.93,1.93,2.01,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
4,2017,4,15,Melbourne,1487214643&p_stn_num=086232,2017-04-15,19:25,Carlton,Gold Coast,Etihad Stadium,...,1.93,1.88,2.00,1.89,1.93,1.94,1.93,1.96,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...


In [19]:
# RETURN THE WEATHER 
def find_rain(row):
    req = requests.get(row.URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # FIND ALL TR TAGS IN TBODY BESIDES ONES WITH A TR CLASS NOT EQUAL TO 'GRAPHCELL'
    Relevent_Data = soup.find('tbody').find_all('tr', class_=lambda x: x != 'graphcell')

    # CALL THE RELEVENT DAY TR LIST WITH INDEX NUMBER (EG. 3rd = index 2)
    Day_Rain = Relevent_Data[(int(row.Day) - 1)]
    
    # FIND ALL THE 'TD' OR MONTH TAGS FOR A GIVEN DAY OF THE MONTH
    All_Months_Rain = Day_Rain.find_all('td')
    
    # ASSIGN THE MONTH OF A GIVEN ROW
    Month_Value = (int(row.Month) - 1)

    # CALL THE RELEVENT MONTH TD LIST WITH INDEX NUMBER (note. we have already created index with necessary index)
    Rain = (All_Months_Rain[Month_Value].string)
    
    return Rain
    #print (Rain)



### APPLY FUNCTION AND RETURN IT TO A 'RAIN' COLUMN BY ROW

In [20]:
df["Rain"] = df[df.City != 'Wellington'].apply(find_rain, axis=1)

### TESTING CODE ON SMALL SAMPLE FIRST

df["Rain"] = df.loc[0].Year.apply(find_rain, axis=1)

df["Rain"] = df[df.City == 'Melbourne'].apply(find_rain, axis=1)

df["Rain"] = df[(df.City == 'Melbourne') & (df.Year == '2017')].apply(find_rain, axis=1)

### TEST A SAMPLE

In [21]:
#df[df['City'] == 'Melbourne'].head(60)
df.head(60)

,Year,Month,Day,City,station_ID,Date,Kick Off (local),Home Team,Away Team,Venue,...,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Unnamed: 51,URL,Rain
0,2017,4,16,Melbourne,1487214643&p_stn_num=086232,2017-04-16,15:20,Hawthorn,Geelong,MCG,...,1.81,2.01,1.97,1.93,1.93,1.94,1.88,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
1,2017,4,16,Brisbane,334797792&p_stn_num=040913,2017-04-16,16:40,Brisbane,Richmond,Gabba,...,1.81,1.96,1.93,1.93,1.93,2.04,1.92,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0
2,2017,4,16,Melbourne,1487214643&p_stn_num=086232,2017-04-16,15:20,Collingwood,St Kilda,Etihad Stadium,...,1.77,1.91,1.91,1.93,1.93,1.94,1.94,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
3,2017,4,15,Adelaide,105924303&p_stn_num=023011,2017-04-15,19:10,Adelaide,Essendon,Adelaide Oval,...,1.85,2.17,1.85,1.93,1.93,1.93,2.01,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0
4,2017,4,15,Melbourne,1487214643&p_stn_num=086232,2017-04-15,19:25,Carlton,Gold Coast,Etihad Stadium,...,1.88,2.00,1.89,1.93,1.94,1.93,1.96,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
5,2017,4,15,Canberra,986923182&p_stn_num=070246,2017-04-15,16:35,GWS Giants,Port Adelaide,Manuka Oval,...,1.81,1.98,1.97,1.93,1.83,1.95,1.88,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
6,2017,4,15,Melbourne,1487214643&p_stn_num=086232,2017-04-15,13:45,Melbourne,Fremantle,MCG,...,1.89,1.99,1.89,1.93,1.94,1.93,1.96,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
7,2017,4,14,Melbourne,1487214643&p_stn_num=086232,2017-04-14,16:20,North Melbourne,Western Bulldogs,Etihad Stadium,...,1.84,1.93,1.89,1.93,1.91,1.93,1.96,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
8,2017,4,13,Perth,16771239&p_stn_num=009151,2017-04-13,18:10,West Coast,Sydney,Domain Stadium,...,1.77,2.16,1.77,1.93,1.83,2.05,2.11,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,None
9,2017,4,9,Gold Coast,326713690&p_stn_num=040416,2017-04-09,16:40,Gold Coast,Hawthorn,Metricon Stadium,...,1.84,1.89,1.85,1.93,1.93,1.96,2.00,NaN,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0


In [25]:
print ("done")

done


In [31]:
file_name = 'TEST2.csv'
df.to_csv(file_name, sep=',', encoding='utf-8')